<a href="https://colab.research.google.com/github/Ssamikarann/Wordle/blob/main/WordleSolution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [97]:
import pandas as pd
import requests
import numpy as np
import string


In [98]:
## Getting relevant words from dictionary

In [99]:



url = "https://norvig.com/ngrams/count_1w.txt"
req = requests.get(url)
if req.status_code == requests.codes.ok:
    given_text = req.text  

In [100]:
word_list = given_text.split()

In [101]:
useful_words = [word for word in word_list if len(word)==5 and word.isdigit()==False]

In [102]:
useful_4_letter_words_list = [word for word in word_list if len(word)==4 and word.isdigit()==False]

In [103]:
#[word for word in useful_words if word[:4] in useful_4_letter_words_list]

In [104]:
#useful_words

## Web Scraping for Archive

In [ ]:
# Before we begin, run this cell if you are using Colab
#!git clone -b 3-ysi-tutorial https://github.com/nestauk/im-tutorials.git

In [ ]:
# Imports
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt

%matplotlib inline

In [105]:
# given homepage
given_url = 'https://www.walesonline.co.uk/whats-on/whats-on-news/wordle-214-previous-answers-tips-22798257'

# Use requests to retrieve data from a given URL
given_response = requests.get(given_url)

# Parse the whole HTML page using BeautifulSoup
given_soup = BeautifulSoup(given_response.text, 'html.parser')

# Title of the parsed page
given_soup.title

<title>Wordle 235: All previous answers, tips and words to begin viral word puzzle - Wales Online</title>

### Get words data

In [106]:

widget_list = given_soup.find_all(attrs={'class':'content-column'})

In [107]:
 all_prev_word_list = widget_list[0].ul.text.split(' #')

In [108]:
df_prev = pd.Series(all_prev_word_list).str.split('-',expand=True)

In [109]:
df_prev = df_prev.rename(columns ={0:'Index_val',1:'Word',2:'Date'})

In [110]:
 if df_prev.iloc[:1]['Word'].isnull().any():
  df_prev = df_prev.iloc[1:]

In [111]:
start_date = df_prev.iloc[-1:]['Date'].values[0]

In [112]:
df_prev['Date'] = pd.date_range(start=start_date,freq='D',periods=len(df_prev)).sort_values(ascending=False)

In [113]:
df_prev['Word'] = df_prev['Word'].str.lower().str.strip()

## Below is testing against the real data

In [115]:
def get_input():
  '''
  [gets input word and the color patter input]
  pattern: g for green , y for yellow , black for black
  
  '''
  input_word = input('WOrd:')

  input_pattern = input('pattern:')
  return input_word,input_pattern

In [123]:
def check_answers(useful_words):
  ''' checks answer after prompting input , input and pattern need to be supplied by user
  args: useful_words: word list from dictionary for suggestions'''
  all_letters = list(string.ascii_lowercase)
  attempts = 6
  all_word = useful_words.copy()

  for i in range(0,attempts):
    word,pattern = get_input()
    if pattern == 'ggggg':
      break
    for loc,(let,pat) in enumerate(zip(word,pattern)):
      if pat =='b':
          if let in all_letters:
            all_letters.remove(let)
            all_word = [word for word in all_word if let not in word]
      elif pat == 'g':
        all_word = [word for word in all_word if word[loc]==let]
      elif pat == 'y':
        all_word = [word for word in all_word if let in word and word[loc]!=let]

    print(all_word[:20])
    



  

    







In [124]:
check_answers(useful_words)

WOrd:adieu
pattern:ggggg


## Test Against any random word, without any input
### Strategy used:

The useful_words listed is sorted by frequency, so use the top element of the available words

In [125]:
def give_pattern(given_word,truth_word):
  out_pat = ''
  for giv_let, tru_let in zip(given_word,truth_word):
    
    if giv_let==tru_let:
      out_pat +='g'
    elif giv_let in truth_word:
      out_pat +='y'
    else:
      out_pat +='b'
  return out_pat



In [126]:
import numpy as np


## Checking against randomly chosen previous word
Strategy:
1. Use Starting word
2. Remove the words that do not fit the criteria anymore
3. Then use the most common word 
4. repeat 2 until attempts maxed out

In [128]:
import string
all_letters = list(string.ascii_lowercase)
attempts = 6
all_word = useful_words.copy()

answer_word = np.random.choice(df_prev['Word'],1)[0]       #this is what we are trying to guess


word = 'adieu' # starting word

for i in range(0,attempts):
  print(f'This is Attempt:{i+1}')
  if word == answer_word:
    print('answer_word is found')
    break
  pattern = give_pattern(word,answer_word)
  
  for loc,(let,pat) in enumerate(zip(word,pattern)):
    if pat =='b':
        if let in all_letters:
          all_letters.remove(let)
          all_word = [word for word in all_word if let not in word]
    elif pat == 'g':
      all_word = [word for word in all_word if word[loc]==let]
    elif pat == 'y':
      all_word = [word for word in all_word if let in word and word[loc]!=let]
  
  print(word,pattern)
  
  word = all_word[0]
  print(all_word[:5])

print('-'*100)
print(f'THe answer is:{answer_word}')


  

    









This is Attempt:1
adieu bbgbb
['which', 'click', 'think', 'still', 'print']
This is Attempt:2
which bbgbb
['still', 'print', 'point', 'going', 'prior']
This is Attempt:3
still yygbb
['trips', 'moist', 'frist', 'brits', 'trims']
This is Attempt:4
trips ybgby
['moist', 'joist', 'nxist', 'foist', 'osint']
This is Attempt:5
answer_word is found
----------------------------------------------------------------------------------------------------
THe answer is:moist


In [129]:
import string





results_dict = {}
for answer_word in df_prev['Word']:
  
  word = 'adieu' # starting word
  all_letters = list(string.ascii_lowercase)
  attempts = 6
  all_word = useful_words.copy()


  for i in range(0,attempts):
    
    if word == answer_word:
      
      break
    pattern = give_pattern(word,answer_word)
    
    for loc,(let,pat) in enumerate(zip(word,pattern)):
      if pat =='b':
          if let in all_letters:
            all_letters.remove(let)
            all_word = [word for word in all_word if let not in word]
      elif pat == 'g':
        all_word = [word for word in all_word if word[loc]==let]
      elif pat == 'y':
        all_word = [word for word in all_word if let in word and word[loc]!=let]
    
    
    
    word = all_word[0]
  
  results_dict.update({answer_word:i})
  

    

    









In [130]:
pd.json_normalize(results_dict).T.reset_index()

,index,0
0,humor,3
1,frame,3
2,elder,2
3,skill,3
4,aloft,3
5,pleat,4
6,shard,3
7,moist,4
8,those,2
9,light,3




# Checking Efficacy of Starting Words


In [ ]:

starting_words_list = ['radio','adieu','crane','stare','audio','roate','roast','ratio','arise','tears']
exp_rep_num = 1000
out_dict = {}

for starting_word in starting_words_list :
  corr_ans = 0
  wrong_ans = []
  attempt_list = []
  for i in range(0,exp_rep_num): # repeating experiment
    all_letters = list(string.ascii_lowercase)
    attempts = 6
    all_word = useful_words.copy()
    word = starting_word
    

    #answer_word = np.random.choice(useful_words,1)[0]
    answer_word = np.random.choice(df_prev['Word'],1)[0]
    for attempt in range(0,attempts):
      
      if word == answer_word:
        
        break
      pattern = give_pattern(word,answer_word)
      
      for loc,(let,pat) in enumerate(zip(word,pattern)):
        if pat =='b':
            if let in all_letters:
              all_letters.remove(let)
              all_word = [word for word in all_word if let not in word]
        elif pat == 'g':
          all_word = [word for word in all_word if word[loc]==let]
        elif pat == 'y':
          all_word = [word for word in all_word if let in word and word[loc]!=let]
      
      
      
      word = all_word[0]

    attempt_list.append(attempt+1)
      
    if word == answer_word:
      corr_ans +=1
    else:
      wrong_ans.append(answer_word)
    all_data_dict ={'attempt':attempt_list,
                    'corr_ans':corr_ans,
                    'wrong_ans':wrong_ans} 
    
  out_dict.update({starting_word:all_data_dict} )
  


  

    











In [ ]:
df_start_results = pd.DataFrame({})

In [ ]:
df_start_results = pd.DataFrame({})
for starting_word in starting_words_list:
  attempts_avg = sum(out_dict[starting_word]['attempt'])/exp_rep_num
  #print(f'The average attempts for {starting_word} is {attempts_avg}')

  corr_pct = 1-(len(out_dict[starting_word]['wrong_ans'])/exp_rep_num)
  #print(f'The percentage of correct answer for  {starting_word} is {corr_pct*100}%')
  thresh_list = []
  thresh_res_list = []
  for thresh_val in range(3,7):
    pct_less_than = sum([i<thresh_val for i in out_dict[starting_word]['attempt']])/len(out_dict[starting_word]['attempt'])
    thresh_list.append(f'Prob_LessThan_{thresh_val}')
    thresh_res_list.append(pct_less_than)
    #print(f'The chance of correct answer being found in  less than  {thresh_val} attempts with starting word: {starting_word} is {pct_less_than*100}%')
  df_results_chk = pd.DataFrame(data=np.array([[starting_word,corr_pct,attempts_avg]+ thresh_res_list]),columns=['StartingWord','PctCorrect','AvgAttempts']+thresh_list)
  df_start_results = df_start_results.append(df_results_chk)

  

In [ ]:
df_start_results = df_start_results.sort_values(by='AvgAttempts',ascending=True).reset_index(drop=True)

In [ ]:

df_start_results

,StartingWord,PctCorrect,AvgAttempts,Prob_LessThan_3,Prob_LessThan_4,Prob_LessThan_5,Prob_LessThan_6
0,roast,1.0,3.929,0.104,0.379,0.719,0.869
1,roate,0.976,3.977,0.071,0.349,0.703,0.9
2,tears,1.0,4.004,0.023,0.362,0.696,0.915
3,radio,1.0,4.023,0.028,0.389,0.656,0.904
4,arise,0.97,4.025,0.025,0.274,0.793,0.883
5,stare,1.0,4.077,0.022,0.351,0.623,0.927
6,ratio,1.0,4.095,0.045,0.303,0.705,0.852
7,crane,1.0,4.119,0.027,0.225,0.753,0.876
8,adieu,1.0,4.182,0.0,0.284,0.625,0.909
9,audio,0.976,4.399,0.018,0.237,0.488,0.858
